# Chapter 2

## Fig 2.04 Exponential decay

In [ ]:
using Plots
Plots.default(linewidth=2)

In [ ]:
plot([t-> 3 * exp(-t) t->3 * exp(-2t) t-> 3 * exp(-3t)], 0.0, 5.0, 
     xlim = (0, 5), ylim=(0, 3.2),
     xlabel="Time", ylabel="Concentration", 
     label = ["exp(-t)" "exp(-2t)" "exp(-3t)"], 
     title= "Figure 2.4"
)

## Fig 2.09 

Numerical Simulation of a metabolic network

In [ ]:
using DifferentialEquations
using Catalyst
using ModelingToolkit
using Plots
Plots.default(linewidth=2)

In [ ]:
# Convenience functions
hill(x, k) = x / (x + k)
hill(x, k, n) = hill(x^n, k^n)

In [ ]:
# Model building
net = @reaction_network begin
    3.0, ∅ --> A
    2.0, A --> B
    2.5, A + B --> C + D
    3.0, C --> ∅
    3.0, D --> ∅
end

In [ ]:
speciesmap(net)

In [ ]:
odesys = convert(ODESystem, net)

In [ ]:
u0 = zeros(4)
tend = 10.0
sol = solve(ODEProblem(net, u0, tend))

In [ ]:
plot(sol, xlims=(0.0, 4.0), ylims=(0.0, 1.0), 
     xlabel="Time (sec)", ylabel="Concentration (mM)", title="Figure 2.09",
     legend=:bottomright)

## Figure 2.11-14 

Model reduction of ODE metabolic networks

In [ ]:
using DifferentialEquations
using Catalyst
using ModelingToolkit
using Plots
Plots.default(linewidth=2)

In [ ]:
fullModel = @reaction_network begin
    k_0, 0 --> A
    (k_1, k_m1), A <--> B
    k_2, B --> 0
end k_0 k_1 k_m1 k_2

In [ ]:
fullSys = convert(ODESystem, fullModel)

In [ ]:
speciesmap(fullModel)

In [ ]:
paramsmap(fullModel)

In [ ]:
params = [0, 9, 12, 2]
u0 = [0.0, 10.0]
tend = 3.0

In [ ]:
sol1full = solve(ODEProblem(fullSys, u0, tend, params))

In [ ]:
plot(sol1full, xlabel="Time (AU)", ylabel="Concentration (AU)", title="Fig. 2.11 (Full model)")

## Figure 2.12 : Rapid equilibrium assumption

In [ ]:
reModel = @reaction_network begin
    (k_0, hill(k_1, k_2, k_m1, 1)), 0 <--> B
end k_0 k_1 k_m1 k_2

In [ ]:
speciesmap(reModel)

In [ ]:
paramsmap(reModel)

In [ ]:
u0re = [sum(u0)]
sol1re = solve(ODEProblem(reModel, u0re, tend, params));

In [ ]:
pl2 = plot(sol1full, line=(:dash, 1),label=["A (full solution)" "B (full solution)"])

fracA = params[3] / (params[3] + params[2])
fracB = 1 - fracA

plot!(pl2, sol1re, idxs=((t, b̃)-> (t, b̃ * fracA), 0, 1), 
        label="A (rapid equilibrium)")
plot!(pl2, sol1re, idxs=((t, b̃)-> (t, b̃ * fracB), 0, 1), 
        label="B (rapid equilibrium)")
plot!(pl2, title="Fig. 2.12 (Rapid equilibrium model)", 
        xlabel="Time (AU)", ylabel="Concentration (AU)")

## Figure 2.13: Rapid equilibrium 

with another set of parameters not suitable for Rapid equilibrium assumption.

In [ ]:
tspan = 3.0
params = [9, 20, 12, 2]
u0 = [8, 4]
u0re = [sum(u0)]
sol2full = solve(ODEProblem(fullModel, u0, tspan, params));
sol2re = solve(ODEProblem(reModel, u0re, tspan, params));

In [ ]:
pl3 = plot(sol2full, line=(:dash, 1),label=["A (full solution)" "B (full solution)"])

fracA = params[3] / (params[3] + params[2])
fracB = 1 - fracA

plot!(pl3, sol2re, idxs=((t, b̃)-> (t, b̃ * fracA), 0, 1), 
        label="A (rapid equilibrium)")
plot!(pl3, sol2re, idxs=((t, b̃)-> (t, b̃ * fracB), 0, 1), 
        label="B (rapid equilibrium)")
plot!(pl3, title="Fig. 2.13 (Rapid equilibrium model)", 
        xlabel="Time (AU)", ylabel="Concentration (AU)")

## Figure 2.14 : QSSA

Quasi-steady state assumption on species A

In [ ]:
qssModel = @reaction_network begin
    (k_0, k_2), 0 <--> B
end k_0 k_2

# The initial conditions in QSSA could be tricky
qss_u0(u0, k0, k1, km1, k2) = (k1 * sum(u0) - k0) / (k1 + km1)

In [ ]:
k0, k1, km1, k2 = params
u0qss = qss_u0(u0, k0, k1, km1, k2)

In [ ]:
paramsmap(qssModel)

In [ ]:
sol2qss = solve(ODEProblem(qssModel, [u0qss], tend, (k0, k2)))

In [ ]:
pl4 = plot(sol2full, line=(:dash, 2), xlims=tspan,
 xlabel="Time (arbitrary units)",
 ylabel="Concentration (arbitrary units)",
 title="Figure 2.14: Ref vs QSSA")

plot!(pl4, sol2qss, label="B (QSSA)", line=(2, :red))
plot!(pl4, sol2qss , idxs= ((t, b) -> (t, (k0 + km1*b)/k1), 0, 1), label="A (QSSA)", line=(2, :blue))

## Problem 2.4.6

In [ ]:
using Plots, DifferentialEquations
Plots.default(linewidth=2)

In [ ]:
# Model
f(u, p, t) = p * (1.0-u)

In [ ]:
p = 1.0
u0 = 0.0
tspan = 10.0

In [ ]:
prob = ODEProblem(f, u0, tspan, p)

In [ ]:
sol = solve(prob)
plot(sol, xlabel="Time (arbitrary units)", ylabel="Concentration (arbitrary units)", legend=:bottomright)

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()